In [7]:
from datasets import load_dataset, Dataset
import pandas as pd

df11 = pd.read_parquet('volttronlog_mass.parquet')
df12= pd.read_parquet('540log_mass.parquet')
df21 = pd.read_parquet('volttronlog.parquet')
df22= pd.read_parquet('540log.parquet')
df=pd.concat([df11,df12,df21,df22],ignore_index=True)
df['response']=df['response'].apply(lambda array: array[0] )
raw=Dataset.from_pandas(df[:500])
raw1=raw.train_test_split(test_size=0.2)
print(raw1)

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt', 'context'],
        num_rows: 400
    })
    test: Dataset({
        features: ['response', 'prompt', 'context'],
        num_rows: 100
    })
})


In [6]:
#print(raw1["train"]["response"][0])
print(raw1["train"]["prompt"][0])
#print(raw1["train"]["context"][0])

provide a detaled technical summery of the system for the system log in this given context given bellow . include the time window, how many agents had events. how many error event during the time period and discribe errors:


Timestamp: 2024-06-17 17:54:59,219
Agent: listeneragent-3.3 878937 [99]
Debug: __main__
Level: INFO
Message: Peer: pubsub, Sender: listeneragent-3.3_1:, Bus: , Topic: heartbeat/listeneragent-3.3_1, Headers: {'TimeStamp': '2024-06-17T22:54:59.216330+00:00', 'min_compatible_version': '3.0', 'max_compatible_version': ''}, Message:
'GOOD'
Timestamp: 2024-06-17 17:55:00,001
Agent: platform_driveragent-4.0 878936 [223]
Debug: platform_driver.driver
Level: DEBUG
Message: fake-campus/fake-building/fake-device next scrape scheduled: 2024-06-17 22:55:05+00:00
Timestamp: 2024-06-17 17:55:00,001
Agent: platform_driveragent-4.0 878936 [227]
Debug: platform_driver.driver
Level: DEBUG
Message: scraping device: fake-campus/fake-building/fake-device
Timestamp: 2024-06-17 17:55:00,

In [8]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
prefix = "summarize:"


def preprocess_function(examples):
    inputs = [ doc for doc in examples["prompt"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["response"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_billsum = raw1.map(preprocess_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map: 100%|██████████| 100/100 [00:00<00:00, 380.91 examples/s]


In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [12]:
import evaluate

rouge = evaluate.load("rouge")
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [13]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [14]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


In [16]:
text = raw1["train"]["prompt"][0]
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10280)
print(tokenizer.batch_decode(outputs)[0])

<pad> ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''</s>


In [17]:
from peft import get_peft_model
from peft import prepare_model_for_kbit_training
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
#model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)


model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953


In [18]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
     weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=100,
    warmup_steps=5,
    fp16=True,
    optim="paged_adamw_8bit",
)

/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    
)

trainer.train()

/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,4.959800,4.933370
2,4.941700,4.929127
3,4.942600,4.920507


/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/torch/nn/parallel/_functions.p

TrainOutput(global_step=3, training_loss=4.9480312665303545, metrics={'train_runtime': 255.2921, 'train_samples_per_second': 4.7, 'train_steps_per_second': 0.012, 'total_flos': 1468021918924800.0, 'train_loss': 4.9480312665303545, 'epoch': 3.0})

In [13]:
training_args = TrainingArguments(
    output_dir="your-name/bigscience/mt0-large-lora",
    learning_rate=1e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
     weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=50,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
)

/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
text = 'hi'

In [25]:
model.eval()
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])


<pad><extra_id_0> hi, hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi hi</s>


In [2]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

checkpoint = "facebook/bart-large-cnn"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

text = " 2024-06-18 01:03:00,001 (platform_driveragent-4.0 878936 [223]) platform_driver.driver DEBUG: fake-campus/fake-building/fake-device next scrape scheduled: 2024-06-18 06:03:05+00:00"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])

/home/sanka/anaconda3/envs/LLMv1/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


</s><s><s>Next scrape scheduled: 2024-06-18 06:03:05+00:00 (platform_driveragent-4.0 878936 [223] platform_driver.driver: fake-campus/fake-building/ fake-device) platform-driver: false-campus.driver.fake-device: false.platform-driver.FakeDevice: fake.campus.fake.device.</s>
